### Steel Plate Defect Detection

__Background__: Steel plate defects are extracted from photos of several faulty steel plates with surface imperfections. Images analysis revealed 27 different features to describe the steel fault. A total of 6 unique types of faults are categorized, with a final category of "other faults" for any type of fault that does not fit into the other specific 6 categories.

__Features__: There are 27 features that are used to predict the steel faults. These features are extracted from steel plate samples. Computer vision can automatically extract some of this information from images or manually extracted with a user inspecting each plate defect or photo of the steel plate.

__Objective__: Develop a neural network in Keras / TensorFlow for a multi-class classification problem. 

Additional Information from [Kaggle](https://www.kaggle.com/uciml/faulty-steel-plates) and [Machine Learning for Engineers](https://apmonitor.com/pds/index.php/Main/SteelPlateFaults).

<img align=left width=400px src='https://apmonitor.com/pds/uploads/Main/steel_plates.png'>

### Load Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# scikit-learn packages
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest,chi2

# keras packages
from keras.models import Sequential
from keras.layers import Dense

### Load data and display a sample

In [ ]:
data = pd.read_csv('http://apmonitor.com/pds/uploads/Main/steel.txt')
data.sample(10)

### Detect Outliers

Detect outliers and remove bad data if present.

In [ ]:
features = data.columns[:-7]
data[features].plot(kind='box',subplots=True,layout=(5,6),figsize=(12,10))
plt.show()

### Show Balance of Labels

In [ ]:
features = data.columns[:-7]
labels = data.columns[-7:]
X = data[features]
y = data[labels]
y.idxmax(axis=1).value_counts().plot(kind='bar',figsize=(12,5))

### Scale data, select features, and split into train and test dataset

Keras also has a validation option in the keras fit method with ''validation_split=0.2''.

In [ ]:
# Scale data
s = MinMaxScaler()
data_s = s.fit_transform(data)
data_s = pd.DataFrame(data_s,columns=data.columns)

# Split data into X and y
features = data.columns[:-7]
labels = data.columns[-7:]
X = data_s[features]
y = data_s[labels]

# Train/test split
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=0.2,shuffle=True)

### Select Best Features

Select the best features for the classification.

In [ ]:
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(X,y)
plt.figure(figsize=(12,4))
plt.bar(x=X.columns,height=fit.scores_,)
plt.xticks(rotation=90);

### Create a one-layer neural network in Keras
- 27 inputs -> [8 hidden nodes] -> 7 outputs
- Use categorical cross entropy as the loss metric
- Use 'softmax' activation in the final layer to return probabilities
- Utilize a validation split and plot the loss function to observe any evidence of overfitting


In [ ]:
# Classification neural network with Keras
model = Sequential()
model.add(Dense(8, input_dim=Xtrain.shape[1], activation='relu'))
model.add(Dense(ytrain.shape[1], activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', \
              optimizer='adam', metrics=['accuracy'])

# Train model
result = model.fit(Xtrain,ytrain,epochs=1000,\
                   validation_split=0.2,verbose=0)

### Plot the Train and Validation Loss Functions

Generate the train and validation loss functions to determine number of epochs and evidence of overfitting.

In [ ]:
plt.semilogy(result.history['loss'],label='loss')
plt.semilogy(result.history['val_loss'],label='val_loss')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epoch')

**Discussion:** Look at the plots for loss. Do you see evidence of overfitting? How many epochs are needed for a good fit? Retrain the model with the proper number of epochs.

### Use test set to analyze model effectiveness

Generate predicted labels and probabilities of those labels.

In [ ]:
# Make predictions and save in dataframe
yp = model.predict(Xtest)
yp = pd.DataFrame(yp,columns=ytest.columns)

# Extract predicted labels and probabilities
predicted_label = yp.idxmax(axis=1)
predicted_prob = yp.max(axis=1)
actual_label = ytest.idxmax(axis=1)

### Display probabilities

Display probabilities of faults with the most likely label highlighted and the actual label displayed.

In [ ]:
yp['Actual fault'] = actual_label.values
yp.style.highlight_max(axis=1)

### Generate confusion matrix

In [ ]:
cm = confusion_matrix(predicted_label,actual_label)
sns.heatmap(cm,annot=True)
plt.savefig('confusion_matrix.png',dpi=600)
plt.show()

### Score and Accuracy

In [ ]:
score1, acc1 = model.evaluate(Xtrain, ytrain)
score2, acc2 = model.evaluate(Xtest, ytest)

### Scikit-Learn Classifiers

Repeat analysis with 8 supervised learning methods. See [Classification Overview](https://apmonitor.com/pds/index.php/Main/ClassificationOverview) for additional information on supervised learning methods. 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.multioutput import MultiOutputClassifier

### Create Models

In [ ]:
ada = AdaBoostClassifier()
lr = LogisticRegression()
nb = GaussianNB()
knn = KNeighborsClassifier()
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()
svc = SVC()
mlp = MLPClassifier(max_iter=10000)
clist = [ada,lr,nb,knn,dt,rf,svc,mlp]
mlist = [None]*8

### Train Models

In [ ]:
for i,m in enumerate(clist):
    mlist[i] = MultiOutputClassifier(m).fit(Xtrain,ytrain)

### Train and Test Set Scores

In [ ]:
x = pd.DataFrame()
n = ['Keras/TF NN']; s1 = [score1]; s2 = [score2]
for i,m in enumerate(mlist):
    n.append(str(m.estimator))
    s1.append(m.score(Xtrain,ytrain))
    s2.append(m.score(Xtest,ytest))

In [ ]:
plt.bar(n,s1,label='Train')
plt.bar(n,s2,width=0.5,label='Test')
plt.xticks(rotation=90)
plt.legend()
plt.show()